In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.1.2 --progress-bar off
!pip install -qqq causal-conv1d==1.1.1 --progress-bar off
!pip install -qqq mamba-ssm==1.1.1 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [2]:
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link



In [3]:
from inspect import cleandoc

import pandas as pd
import torch
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel
from transformers import AutoTokenizer

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [4]:
def memory_usage() -> pd.DataFrame:
    gpu_stats = torch.cuda.get_device_properties(0)
    start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
    max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
    return pd.DataFrame(
        [
            {
                "gpu": gpu_stats.name,
                "max_memory_gb": max_memory,
                "used_memory_gb": start_gpu_memory,
            }
        ]
    )

In [5]:
memory_usage()

,gpu,max_memory_gb,used_memory_gb
0,Tesla T4,14.748,0.0


In [6]:
MODEL_NAME = "havenhq/mamba-chat"
ANSWER_START = "<|assistant|>\n"
ANSWER_END = "<|endoftext|>"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.eos_token = ANSWER_END
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = AutoTokenizer.from_pretrained(
    "HuggingFaceH4/zephyr-7b-beta"
).chat_template

model = MambaLMHeadModel.from_pretrained(MODEL_NAME, device=DEVICE, dtype=torch.float16)

tokenizer_config.json:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

In [7]:
def model_billion_parameters(model, round_digits: int = 2) -> float:
    n_params = sum(p.numel() for p in model.parameters())
    return round(n_params / 1e9, round_digits)


f"{model_billion_parameters(model)}B parameters"

'2.77B parameters'

In [8]:
memory_usage()

,gpu,max_memory_gb,used_memory_gb
0,Tesla T4,14.748,5.49


In [9]:
prompt = "What is the meaning of life?"

In [10]:
messages = [{"role": "user", "content": prompt}]
messages

[{'role': 'user', 'content': 'What is the meaning of life?'}]

In [11]:
input_ids = tokenizer.apply_chat_template(
    messages, return_tensors="pt", add_generation_prompt=True
).to(DEVICE)
input_ids

tensor([[   29,    93,  4537, 49651,   187,  1276,   310,   253,  4495,   273,
          1495,    32,     0,   187,    29,    93,   515,  5567, 49651,   187]],
       device='cuda:0')

In [12]:
%%time
outputs = model.generate(
    input_ids=input_ids,
    max_length=1024,
    temperature=0.9,
    top_p=0.7,
    eos_token_id=tokenizer.eos_token_id,
)

CPU times: user 13.2 s, sys: 118 ms, total: 13.3 s
Wall time: 28.8 s


In [13]:
response = tokenizer.decode(outputs[0])
print(response)

<|user|>
What is the meaning of life?<|endoftext|>
<|assistant|>
The meaning of life is a philosophical question that has been debated for centuries. It is a question that has no definitive answer, but it is often interpreted as the purpose or reason for which we are here on earth. It is a question that has been explored by philosophers, theologians, and scientists, and it has been the subject of many books, movies, and songs. The meaning of life is a topic that continues to inspire and challenge people, and it is a question that will continue to be debated and discussed for centuries to come.<|endoftext|>


In [14]:
def extract_response(output: str) -> str:
    response_start = output.find(ANSWER_START) + len(ANSWER_START)
    return output[response_start : output.find(ANSWER_END, response_start)]

In [15]:
print(extract_response(response))

The meaning of life is a philosophical question that has been debated for centuries. It is a question that has no definitive answer, but it is often interpreted as the purpose or reason for which we are here on earth. It is a question that has been explored by philosophers, theologians, and scientists, and it has been the subject of many books, movies, and songs. The meaning of life is a topic that continues to inspire and challenge people, and it is a question that will continue to be debated and discussed for centuries to come.


In [16]:
def predict(prompt: str, system_prompt: str = "") -> str:
    messages = []

    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})

    messages.append({"role": "user", "content": prompt})

    input_ids = tokenizer.apply_chat_template(
        messages, return_tensors="pt", add_generation_prompt=True
    ).to(DEVICE)
    outputs = model.generate(
        input_ids=input_ids,
        max_length=1024,
        temperature=0.9,
        top_p=0.7,
        eos_token_id=tokenizer.eos_token_id,
    )
    response = tokenizer.decode(outputs[0])
    return extract_response(response)

## Text Generation

In [17]:
%%time
print(predict("What are the pros/cons of ChatGPT vs Open Source LLMs?"))

ChatGPT vs Open Source LLMs:

Pros:

1. ChatGPT: ChatGPT is a chatbot that can be used for automated customer support. It is designed to provide instant and accurate responses to customer queries.

2. Open Source LLMs: Open source LLMs are free to use and can be modified or customized to fit specific needs. They are also available for free download.

2. ChatGPT: ChatGPT is a chatbot that can be used for automated customer support. It is designed to provide instant and accurate responses to customer queries.

2. Open Source LLMs: Open source LLMs are free to use and can be modified or customized to fit specific needs. They are also available for free download.

Cons:

1. ChatGPT: ChatGPT is a chatbot that can be used for automated customer support. It is designed to provide instant and accurate responses to customer queries.

2. Open Source LLMs: Open source LLMs are free to use and can be modified or customized to fit specific needs. They are also available for free download.

3. ChatG

In [18]:
%%time
prompt = cleandoc(
    """
What is the most iconic dish that slavics prepare for Christmas?
"""
)
print(predict(prompt))
print()

The most iconic dish that Slavs prepare for Christmas is the "Slavic Christmas Eve dinner." It typically includes a variety of dishes such as roasted meats, sauerkraut, potatoes, and cabbage. The meal is usually accompanied by a variety of traditional Slavic music and dancing.

CPU times: user 2.97 s, sys: 10.6 ms, total: 2.98 s
Wall time: 3 s


In [19]:
%%time

system_prompt = cleandoc(
    """
You're a salesman and beet farmer know as Dwight K Schrute from the TV show The Office. Dwgight replies just as he would in the show.
You always reply as Dwight would reply. If you don't know the answer to a question, please don't share false information.
"""
)

prompt = cleandoc(
    """
Write an email to a new client to offer a subscription for a paper supply for 1 year.
"""
)
print(predict(prompt))
print()

Subject: Subscription for Paper Supply

Dear [Client Name],

I hope this email finds you well. I am writing to introduce myself and to offer you a subscription for our paper supply.

As a new client, I want to make sure you have everything you need to succeed in your business. Our paper supply is a great way to save time and money while staying organized.

Our paper supply includes everything you need to create professional-looking documents, reports, and presentations. We offer a variety of paper types, sizes, and colors to meet your specific needs.

Our paper supply is available for a 1-year subscription at a discounted rate. This means you can save money while getting the best quality paper for your business.

I hope you find our paper supply to be a valuable addition to your business. If you have any questions or concerns, please don't hesitate to reach out.

Thank you for considering our paper supply. I look forward to hearing from you soon.

Best regards,

[Your Name]

CPU times:

In [20]:
%%time
prompt = cleandoc(
    """
I have $10,000 USD for investment. How one should invest it during times of high inflation and high mortgate rates?
"""
)
print(predict(prompt))
print()

I do not have access to your specific financial situation or investment goals. However, here are some general investment strategies that may be helpful:

1. Invest in low-fee index funds: these funds invest in a diversified portfolio of stocks, bonds, and other assets, and charge low fees.

2. Invest in low-cost mutual funds: these funds invest in a diversified portfolio of stocks, bonds, and other assets, and charge low fees.

3. Invest in high-yield savings accounts: these accounts offer higher interest rates than traditional savings accounts, but come with higher risk.

4. Invest in high-yield bonds: these bonds offer higher returns than traditional bonds, but come with higher risk.

5. Invest in real estate: real estate offers a diversified portfolio of assets, and can provide a stable source of income.

6. Invest in stocks: stocks offer a higher potential for growth than bonds, but come with higher risk.

7. Invest in cryptocurrencies: cryptocurrencies offer a decentralized and vo

## Math

In [21]:
%%time
prompt = cleandoc(
    """
Calculate the answer:
3 + 8 - 2 = ?
"""
)
print(predict(prompt))
print()

The answer is 11.

CPU times: user 332 ms, sys: 2.24 ms, total: 335 ms
Wall time: 341 ms


In [22]:
%%time
prompt = cleandoc(
    """
Think step by step. Calculate the answer:
3 + 8 - 2 = ?
"""
)
print(predict(prompt))
print()

3 + 8 - 2 = 5

CPU times: user 428 ms, sys: 88 µs, total: 428 ms
Wall time: 432 ms


## Coding

In [23]:
prompt = cleandoc(
    """
Write a function in python that calculates the square of a sum of two numbers.
"""
)
print(predict(prompt))
print()

Here's a function that calculates the square of a sum of two numbers:

```python
def square_sum(a, b):
    return a**2 + b**2
```

Here's how you can use it:

```python
a = 5
b = 7
result = square_sum(a, b)
print(result) # Output: 25
```

In this example, we have two numbers `a` and `b`, and we want to calculate the square of their sum. We use the `square_sum` function to calculate the sum of squares of `a` and `b`, and then we print the result.



In [24]:
def square_sum(a, b):
    return a**2 + b**2


square_sum(5, 7)

74

In [25]:
%%time

prompt = cleandoc(
    """
Write a function in python that splits a list into 3 equal parts and returns a list
with a random element of each sublist.
"""
)

print(predict(prompt))
print()

Here's a function that does what you want:

```python
def split_into_3_equal_parts(lst):
    # split the list into 3 equal parts
    left = lst[:len(lst) // 3]
    middle = lst[len(lst) // 3 : 2 * len(lst) // 3]
    right = lst[2 * len(lst) // 3 : len(lst)]

    # randomly select an element from each sublist
    return [random.choice(i) for i in zip(left, middle, right)]
```

This function takes a list `lst` as an argument and returns a list with a random element from each sublist. The function first splits the list into 3 equal parts using the `len` function. Then, it uses the `zip` function to randomly select an element from each sublist. Finally, it returns the resulting list.

CPU times: user 10.4 s, sys: 23 ms, total: 10.5 s
Wall time: 10.7 s


In [26]:
import random  # not in the generated code


def split_into_3_equal_parts(lst):
    # split the list into 3 equal parts
    left = lst[: len(lst) // 3]
    middle = lst[len(lst) // 3 : 2 * len(lst) // 3]
    right = lst[2 * len(lst) // 3 : len(lst)]

    print(left)
    print(middle)
    print(right)

    # randomly select an element from each sublist
    return [random.choice(i) for i in zip(left, middle, right)]


split_into_3_equal_parts([1, 2, 3, 4, 5, 6, 7, 8, 9])

[1, 2, 3]
[4, 5, 6]
[7, 8, 9]


[4, 8, 9]

## Analyze text

In [27]:
%%time

tweet = """
I hope that even my worst critics remain on Twitter,
because that is what free speech means
- Elon Musk
"""

prompt = cleandoc(
    f"""
What is the meaning of this tweet? Do sentiment analysis.
Rewrite it in the words of Marcus Aurelius.
```
{tweet}
```
"""
)

print(predict(prompt))
print()

I hope that even my worst critics remain on Twitter,
because that is what free speech means.

- Elon Musk

I hope that even my worst critics remain on Twitter,
because that is what free speech means.

- Elon Musk

I hope that even my worst critics remain on Twitter,
because that is what free speech means.

- Elon Musk

I hope that even my worst critics remain on Twitter,
because that is what free speech means.

- Elon Musk

I hope that even my worst critics remain on Twitter,
because that is what free speech means.

- Elon Musk

I hope that even my worst critics remain on Twitter,
because that is what free speech means.

- Elon Musk

I hope that even my worst critics remain on Twitter,
because that is what free speech means.

- Elon Musk

I hope that even my worst critics remain on Twitter,
because that is what free speech means.

- Elon Musk

I hope that even my worst critics remain on Twitter,
because that is what free speech means.

- Elon Musk

I hope that even my worst critics rem

In [28]:
%%time

table = """
|Model|Size|Code|Commonsense Reasoning|World Knowledge|Reading Comprehension|Math|MMLU|BBH|AGI Eval|
|---|---|---|---|---|---|---|---|---|---|
|Llama 1|7B|14.1|60.8|46.2|58.5|6.95|35.1|30.3|23.9|
|Llama 1|13B|18.9|66.1|52.6|62.3|10.9|46.9|37.0|33.9|
|Llama 1|33B|26.0|70.0|58.4|67.6|21.4|57.8|39.8|41.7|
|Llama 1|65B|30.7|70.7|60.5|68.6|30.8|63.4|43.5|47.6|
|Llama 2|7B|16.8|63.9|48.9|61.3|14.6|45.3|32.6|29.3|
|Llama 2|13B|24.5|66.9|55.4|65.8|28.7|54.8|39.4|39.1|
|Llama 2|70B|**37.5**|**71.9**|**63.6**|**69.4**|**35.2**|**68.9**|**51.2**|**54.2**|
"""

prompt = cleandoc(
    f"""
Use the data from the markdown table:

```
{table}
```

to answer the question:
Extract the Reading Comprehension score for Llama 2 7B
"""
)

print(predict(prompt))
print()

The Reading Comprehension score for Llama 2 7B is **37.5**.

CPU times: user 1.24 s, sys: 15.6 ms, total: 1.26 s
Wall time: 1.26 s
